In [ ]:
# CIFAR-10 Classification with Improved CNN

# ## 1. Data Preparation

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load data
#dataset: set of 60,000 color images (32x32 pixels) -> 10 classes
(x_train, y_train), (x_test, y_test) = cifar10.load_data()   #load data (already split into 4 parts - x means training images while y is categories for images)


# Normalize -> change pxel values range -> 0-255 to 0-1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode
'''
Before :
y_train = [6, 2, 3]

After :
each matix row -> a spf sample
each matrix column -> a spf category
y_train = [
  [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],  # 6 → frog
  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 2 → bird
  [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]   # 3 → cat
]

'''
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Optional: Augmentation
# This creates an ImageDataGenerator object that will
# randomly transform images during training to simulate variations and prevent overfitting.
datagen = ImageDataGenerator(
    rotation_range=15, # Randomly rotates images by up to ±15 degrees
    width_shift_range=0.1, # Randomly shifts images horizontally by up to 10% of image width
    height_shift_range=0.1, # Randomly shifts images vertically by up to 10% of image height
    horizontal_flip=True # Randomly flips images horizontally (left↔right)
)
datagen.fit(x_train)

# ## 2. Baseline Model

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

baseline_model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)), # Applies 32 filters of size 3x3 over the input image
    MaxPooling2D((2,2)), # Downsamples the feature maps by taking the max value in 2x2 patches
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    Flatten(), # Flattens the 2D feature maps into a 1D vector to feed into the Dense (fully connected) layers.
    Dense(64, activation='relu'), # Fully connected layer with 64 neurons
    Dense(10, activation='softmax') # layer with 10 units (since CIFAR-10 has 10 classes).
                                    # softmax activation: outputs a probability distribution over the 10 classes.
])

baseline_model.compile(optimizer='adam',
                       loss='categorical_crossentropy', # Since your labels are one-hot encoded, this is the correct loss function.
                                                        # It measures the difference between the predicted probability distribution
                                                        # (from softmax) and the actual distribution (one-hot vector).
                       metrics=['accuracy'])            # percentage of correctly predicted classes.

baseline_history = baseline_model.fit(x_train, y_train, # Your training data and labels (features + one-hot labels).
                                      validation_data=(x_test, y_test), # After each epoch, the model evaluates its performance on test data to monitor generalization.
                                      epochs=10, # The model will go through the entire training dataset 10 times.
                                      batch_size=64) # each epoch will process 50,000 training samples but The data is processed in mini-batches of 64 samples at a time
                                                     # 50000 / 64 ≈ 781.25 → rounded to 782 batches -> The model will run 782 mini-updates (batches) per epoch.

# ## 3. Improved Model

from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

model = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    BatchNormalization(),
    Conv2D(32, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),

    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D(),
    Dropout(0.25),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True),
    ReduceLROnPlateau(patience=5, factor=0.5)
]

history = model.fit(datagen.flow(x_train, y_train, batch_size=64),
                    validation_data=(x_test, y_test),
                    epochs=100,
                    callbacks=callbacks)

# ## 4. Evaluation

# Plot accuracy/loss
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.title("Accuracy Curve")
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.legend()
plt.title("Loss Curve")
plt.show()

# Confusion matrix and report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred_classes))


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 87s 107ms/step - accuracy: 0.3480 - loss: 1.7821 - val_accuracy: 0.5382 - val_loss: 1.2974
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 139s 104ms/step - accuracy: 0.5647 - loss: 1.2323 - val_accuracy: 0.6066 - val_loss: 1.1314
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 72s 91ms/step - accuracy: 0.6277 - loss: 1.0682 - val_accuracy: 0.6295 - val_loss: 1.0626
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 90ms/step - accuracy: 0.6664 - loss: 0.9673 - val_accuracy: 0.6697 - val_loss: 0.9600
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 81s 89ms/step - accuracy: 0.6942 - loss: 0.8873 - val_accuracy: 0.6678 - val_loss: 0.9592
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 99ms/step - accuracy: 0.7077 - loss: 0.8454 - val_accuracy: 0.6784 - val_loss: 0.9311
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 90s 109ms/step - accuracy: 0.7230 - loss: 0.8074 - val_accuracy: 0.6894 - val_loss: 0.9050
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 126s 89ms/step - accuracy: 0.7394 - loss: 0.75

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 507ms/step - accuracy: 0.3339 - loss: 2.0720

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 442s 565ms/step - accuracy: 0.4970 - loss: 1.3947 - val_accuracy: 0.6134 - val_loss: 1.1064 - learning_rate: 0.0010
Epoch 3/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 524ms/step - accuracy: 0.5685 - loss: 1.2104

782/782 ━━━━━━━━━━━━━━━━━━━━ 428s 547ms/step - accuracy: 0.5685 - loss: 1.2103 - val_accuracy: 0.6470 - val_loss: 1.0422 - learning_rate: 0.0010
Epoch 4/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.6095 - loss: 1.1051

782/782 ━━━━━━━━━━━━━━━━━━━━ 410s 524ms/step - accuracy: 0.6095 - loss: 1.1051 - val_accuracy: 0.6615 - val_loss: 1.0398 - learning_rate: 0.0010
Epoch 5/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 501ms/step - accuracy: 0.6456 - loss: 1.0235

782/782 ━━━━━━━━━━━━━━━━━━━━ 408s 522ms/step - accuracy: 0.6456 - loss: 1.0235 - val_accuracy: 0.6938 - val_loss: 0.8920 - learning_rate: 0.0010
Epoch 6/100
782/782 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.6630 - loss: 0.9767

782/782 ━━━━━━━━━━━━━━━━━━━━ 410s 524ms/step - accuracy: 0.6630 - loss: 0.9767 - val_accuracy: 0.7096 - val_loss: 0.8671 - learning_rate: 0.0010
Epoch 7/100
 16/782 ━━━━━━━━━━━━━━━━━━━━ 6:55 543ms/step - accuracy: 0.6633 - loss: 1.0089